## Constructing light curves using python - Kepler 5 system

In [4]:
from platform import python_version
print(python_version())

3.7.6


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objects as go
import plotly.express as px
import cufflinks as cf
init_notebook_mode(connected = True)
cf.go_offline()

# Transit Method

### Data cleaning and preparation of the Kepler_5b Time Series dataset.

### Extraction of Kepler_5b Time series dataset.

In [2]:
k = open('Kepler_5b.txt', 'r')
kep = k.read()

In [3]:
read_file = pd.read_csv('Kepler_5b.txt')
Kep =  read_file.to_csv('Kepler_5b.csv', index = None)
KEP = pd.read_csv('Kepler_5b.csv')
KEP

,#FILENAME= 'hlsp_exo_kepler_phot_KPLR8191672_kep_v1.0_dtr.txt'
0,#TARGNAME= 'KPLR8191672'
1,#TELESCOP= 'Kepler' / Telescope use...
2,#INSTRUME= 'Photometer' / Instrument us...
3,#RA_TARG = 299.40700 / [deg] target ...
4,#DEC_TARG= 44.03503 / [deg] target ...
...,...
2114,2454997.901531 0.999957
2115,2454997.921966 0.999833
2116,2454997.942400 0.999837
2117,2454997.962835 1.000386


In [4]:
KEP_c = KEP.head(17)
KEP_cc = KEP_c.rename(columns = {"#FILENAME= 'hlsp_exo_kepler_phot_KPLR8191672_kep_v1.0_dtr.txt'" :"col"})
KEP_cc

,col
0,#TARGNAME= 'KPLR8191672'
1,#TELESCOP= 'Kepler' / Telescope use...
2,#INSTRUME= 'Photometer' / Instrument us...
3,#RA_TARG = 299.40700 / [deg] target ...
4,#DEC_TARG= 44.03503 / [deg] target ...
5,#RMAGTARG= 13.321 / target r magn...
6,#G-R_TARG= 0.562 / target g-r ma...
7,#DATEMIN = 2454953.538373 / earliest obse...
8,#DATEMAX = 2454997.983269 / latest observ...
9,#CAD_TIME= 1765.4976 / [s] median ti...


In [5]:
KEP_d = KEP.tail(2102)
KEP_d
KEP_dc = KEP_d.rename(columns = {"#FILENAME= 'hlsp_exo_kepler_phot_KPLR8191672_kep_v1.0_dtr.txt'" :"col"})
KEP_dc

,col
17,2454953.538373 1.000174
18,2454953.558807 1.000038
19,2454953.579242 1.000179
20,2454953.599676 1.000335
21,2454953.620111 1.000013
...,...
2114,2454997.901531 0.999957
2115,2454997.921966 0.999833
2116,2454997.942400 0.999837
2117,2454997.962835 1.000386


In [6]:
KEP_dcc1 = KEP_dc['col'].str[0:15]
KEP_dc1 = KEP_dcc1.rename('HJD').astype('float')
KEP_dc1

17      2.454954e+06
18      2.454954e+06
19      2.454954e+06
20      2.454954e+06
21      2.454954e+06
            ...     
2114    2.454998e+06
2115    2.454998e+06
2116    2.454998e+06
2117    2.454998e+06
2118    2.454998e+06
Name: HJD, Length: 2102, dtype: float64

In [7]:
KEP_dcc2 = KEP_dc['col'].str[16:25]
KEP_dc2 = KEP_dcc2.rename('Detr_Norm_Flux').astype('float')
KEP_dc2

17      1.000174
18      1.000038
19      1.000179
20      1.000335
21      1.000013
          ...   
2114    0.999957
2115    0.999833
2116    0.999837
2117    1.000386
2118    0.999854
Name: Detr_Norm_Flux, Length: 2102, dtype: float64

In [8]:
kepler = pd.concat([KEP_dc1,KEP_dc2], axis = 1)
kepler.index = kepler.index - 16
kepler

,HJD,Detr_Norm_Flux
1,2.454954e+06,1.000174
2,2.454954e+06,1.000038
3,2.454954e+06,1.000179
4,2.454954e+06,1.000335
5,2.454954e+06,1.000013
...,...,...
2098,2.454998e+06,0.999957
2099,2.454998e+06,0.999833
2100,2.454998e+06,0.999837
2101,2.454998e+06,1.000386


In [9]:
kepler

,HJD,Detr_Norm_Flux
1,2.454954e+06,1.000174
2,2.454954e+06,1.000038
3,2.454954e+06,1.000179
4,2.454954e+06,1.000335
5,2.454954e+06,1.000013
...,...,...
2098,2.454998e+06,0.999957
2099,2.454998e+06,0.999833
2100,2.454998e+06,0.999837
2101,2.454998e+06,1.000386


In [10]:
min(kepler['Detr_Norm_Flux'])

0.992533

### Plot without considering the modulo function

In [11]:
K = px.scatter(kepler, x = 'HJD', y = 'Detr_Norm_Flux', title = 'Light curve - Kepler 5b system')
K.update(layout=dict(title=dict(x=0.5)))

In [12]:
period = 3.5485
kepler['moduloJD'] = kepler['HJD']%period
kepler

,HJD,Detr_Norm_Flux,moduloJD
1,2.454954e+06,1.000174,1.880373
2,2.454954e+06,1.000038,1.900807
3,2.454954e+06,1.000179,1.921242
4,2.454954e+06,1.000335,1.941676
5,2.454954e+06,1.000013,1.962111
...,...,...,...
2098,2.454998e+06,0.999957,0.113031
2099,2.454998e+06,0.999833,0.133466
2100,2.454998e+06,0.999837,0.153900
2101,2.454998e+06,1.000386,0.174335


In [13]:
kepler_5b = kepler.rename(columns = {"moduloJD" :"Phase (in days)",
                                     "Detr_Norm_Flux" : "Normalized Flux"})

# Kepler - 5b Folded light curve 
### Phase (in days) vs Normalized Flux

In [14]:
K = px.scatter(kepler_5b, x = "Phase (in days)", y = "Normalized Flux", title = 'Folded light curve - Kepler 5b system')
K.update(layout=dict(title=dict(x=0.5)))

# Radial Velocity Method
### Data Cleaning and preparation of the Kepler - 5b Ground Based Time Series dataset

### Extraction of Kepler - 5b Ground based time series dataset 

In [ ]:
K = open('Kepler_5b_rv.txt')
K_ep = K.read()
print(K_ep)

In [16]:
readfile = pd.read_csv('Kepler_5b_rv.txt')
Ke_p =  readfile.to_csv('Kepler_5b_rv.csv', index = None)
K_E_P = pd.read_csv('Kepler_5b_rv.csv')
K_E_P

,#FILENAME= 'hlsp_exo_kepler_spec_KPLR8191672_wide_v1.0_rvb.txt'
0,#TARGNAME= 'KPLR8191672'
1,#RA_TARG = 299.40700 / [deg] target ...
2,#DEC_TARG= 44.03503 / [deg] target ...
3,#RMAGTARG= 13.321 / target r magn...
4,#G-R_TARG= 0.562 / target g-r ma...
5,#DATEMIN = 2454987.097346 / earliest obse...
6,#DATEMAX = 2455110.814132 / latest observ...
7,#
8,#NPTS = 8 / number of tim...
9,#COLUMN_1= 'BJD' / Barycentric J...


In [17]:
K_E_p = K_E_P.tail(8)
K_e_P = K_E_p["#FILENAME= 'hlsp_exo_kepler_spec_KPLR8191672_wide_v1.0_rvb.txt'"].str[0:23]
k_e_Pjd = K_E_p["#FILENAME= 'hlsp_exo_kepler_spec_KPLR8191672_wide_v1.0_rvb.txt'"].str[0:14].astype('float').rename('JD')
k_e_Pjd
k_e_Prv = K_E_p["#FILENAME= 'hlsp_exo_kepler_spec_KPLR8191672_wide_v1.0_rvb.txt'"].str[16:23].astype('float').rename('RV')
k_e_Prv

16    164.10
17   -162.44
18   -239.65
19    114.26
20     46.39
21   -263.88
22    167.19
23    172.90
Name: RV, dtype: float64

In [18]:
KEPLER_RV = pd.concat([k_e_Pjd,k_e_Prv], axis = 1)
KEPLER_RV.index = KEPLER_RV.index - 15
KEPLER_RV['moduloJD'] = KEPLER_RV['JD']%period
KEPLER_RV

,JD,RV,moduloJD
1,2.454987e+06,164.10,3.502846
2,2.454988e+06,-162.44,0.944549
3,2.454989e+06,-239.65,1.886185
4,2.455015e+06,114.26,2.939947
5,2.455016e+06,46.39,0.492323
6,2.455017e+06,-263.88,1.545975
7,2.455076e+06,167.19,3.502062
8,2.455111e+06,172.90,3.022132


In [19]:
KEPLER_RV.info()
KEPLER_RV['RV'].min()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 1 to 8
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   JD        8 non-null      float64
 1   RV        8 non-null      float64
 2   moduloJD  8 non-null      float64
dtypes: float64(3)
memory usage: 324.0 bytes


-263.88

In [20]:
KEPLER_RV_5b = KEPLER_RV.rename(columns = {"moduloJD" :"Phase (in days)",
                                           "RV" : "Radial Velocity (m/s)"})


In [21]:
KEPLER_RV_5b.to_csv('Kepler-5b-rv.csv')

# Kepler - 5b Radial Velocity 
### Phase (in days) vs Radial Velocity (m/s)

In [22]:
K_RV=px.scatter(KEPLER_RV_5b,x="Phase (in days)",y="Radial Velocity (m/s)",title='Radial Velocity Method',size="JD")
K_RV.update(layout=dict(title=dict(x=0.5)))